In [5]:
import numpy as np
from pose_evaluation_utils import * 
import quaternion
from scipy.spatial.transform import Rotation as R

rot2quat --> correct (1 test)
quat2mat --> correct (1 test) ???
mat2euler --> correct (1 test)
euler2mat --> correct (1 test)
euler2quat --> correct (1 test)

In [28]:
# rot2quat
R_test = [[ 0.92936118,  0.,         -0.36917178],
         [ 0.,          1.,          0.],
         [ 0.36917178,  0.,          0.92936118]]

q_approx = rot2quat(R_test)
q_func = R.from_matrix(R_test).as_quat()

print('DF-VO:\n {}\n\nScipy:\n {}'.format(q_approx, q_func))

DF-VO:
 (0.9821815456405263, 0.0, -0.1879345934179944, 0.0)

Scipy:
 [ 0.         -0.18793459  0.          0.98218155]


In [23]:
# quat2mat
q_test = [0.0, 0.18793459236621857, 0.0, -0.9821815490722656]
q_test_wxyz = [-0.9821815490722656, 0.0, 0.18793459236621857, 0.0]

R_approx = quat2mat(q_test_wxyz)
R_func = R.from_quat(q_test).as_matrix()
print('DF-VO:\n {}\n\nScipy:\n {}'.format(R_approx, R_func))

DF-VO:
 [[ 0.92936118  0.         -0.36917178]
 [ 0.          1.          0.        ]
 [ 0.36917178  0.          0.92936118]]

Scipy:
 [[ 0.92936118  0.         -0.36917178]
 [ 0.          1.          0.        ]
 [ 0.36917178  0.          0.92936118]]


In [18]:
# mat2euler
eul_approx = mat2euler(R_test)
eul_func = R.from_matrix(R_test).as_euler('zyx')
print('DF-VO:\n {}\n\nScipy:\n {}'.format(eul_approx, eul_func))

DF-VO:
 (-0.0, -0.37811769029318026, -0.0)

Scipy:
 [ 0.         -0.37811769  0.        ]


In [21]:
# euler2mat
eul_test = [0.0, -0.37811769029318026, 0.0]

R_fe_approx = euler2mat(*eul_test)
R_fe_func = R.from_euler('zyx', eul_test).as_matrix()
print('DF-VO:\n {}\n\nScipy:\n {}'.format(R_fe_approx, R_fe_func))

DF-VO:
 [[ 0.92936118  0.         -0.36917178]
 [ 0.          1.          0.        ]
 [ 0.36917178  0.          0.92936118]]

Scipy:
 [[ 0.92936118 -0.         -0.36917178]
 [ 0.          1.         -0.        ]
 [ 0.36917178  0.          0.92936118]]


In [22]:
# euler2quat
q_fe_approx = euler2quat(*eul_test)
q_fe_func = R.from_euler('zyx', eul_test).as_quat()
print('DF-VO:\n {}\n\nScipy:\n {}'.format(q_fe_approx, q_fe_func))

DF-VO:
 [ 0.98218155  0.         -0.18793459  0.        ]

Scipy:
 [ 0.         -0.18793459  0.          0.98218155]


In [34]:
q_test2 = rot2quat(quat2mat(q_test_wxyz))
print('Before:\n {}\n\nAfter:\n {}'.format(q_test_wxyz, q_test2))

Before:
 [-0.9821815490722656, 0.0, 0.18793459236621857, 0.0]

After:
 (0.9821815459558763, 0.0, -0.18793459176991598, 0.0)


In [59]:
q_test0 = np.array([0.0, -0.9821815459558763, 0.0, 0.18793459176991603])
q_test3 = R.from_matrix(R.from_quat(q_test0).as_matrix()).as_quat()
print('Before:\n {}\n\nAfter:\n {}'.format(q_test0, q_test3))

Before:
 [ 0.         -0.98218155  0.          0.18793459]

After:
 [ 0.          0.98218155  0.         -0.18793459]
